<a href="https://colab.research.google.com/github/adrincont/Tareas_UNAL/blob/main/Ejemplo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplo